In [1]:
import cv2     
import math 
import matplotlib.pyplot as plt   
%matplotlib inline
import pandas as pd
from keras.preprocessing import image 
import numpy as np   
from keras.utils import np_utils
import os
import numpy as np
import cv2
from glob import glob
from skimage.measure import compare_mse, compare_nrmse, compare_ssim, compare_psnr
import PIL
import warnings
warnings.filterwarnings('ignore')
import csv

Extracting frames from a video file

In [2]:
def video_to_frames(input_loc, output_loc):
    time_start = time.time()
    cap = cv2.VideoCapture(input_loc)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
        grey_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), grey_image)
        count = count + 1
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds forconversion." % (time_end-time_start))
            break

Calculating intensity of frames in the form of histograms 

In [3]:
def get_histogram(video_path):
    cap = cv2.VideoCapture(video_path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count=0
    video_intensity=[]
    while True:
        ret, frame = cap.read()
        grey_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        histogram = cv2.calcHist([grey_image], [0], None, [256], [0, 255])
        video_intensity.append(histogram.mean())
        count = count + 1     
        if (count > (video_length-1)):
            cap.release()
            return video_intensity
            break
            

Calculating energy and entropy

In [4]:
def energy_entropy(video_path):
    cap = cv2.VideoCapture(video_path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count=0
    energy=[]
    entropy=[]

    while True:
        ret, frame = cap.read()
        grey_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img_arr=np.array(grey_image)
        feat_lbp=local_binary_pattern(img_arr,8,1,'uniform')
        lbp_img=PIL.Image.fromarray(feat_lbp)
        lbp_hist=np.histogram(feat_lbp,8)
        lbp_hist=np.array(lbp_hist[0],dtype=float)
        lbp_prob=np.divide(lbp_hist,np.sum(lbp_hist))
        lbp_energy=round(np.sum(lbp_prob**2),3)
        lbp_entropy=round(-np.sum(np.multiply(lbp_prob,np.log2(lbp_prob))),3)
        energy.append(lbp_energy)
        entropy.append(lbp_entropy)
    
        count = count + 1     
        if (count > (video_length-1)):
            cap.release()
            return energy,entropy
            break
    

In [5]:
def getMse(img1,img2):
    return round(compare_mse(img1,img2),3)

def getPSNR(img1,img2):
    return round(compare_psnr(img1,img2),3)

def getHistCompare(img1,img2):
    hist1 = cv2.calcHist([img1],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([img2],[0],None,[256],[0,256])
    #Histogram Bhattacharya Distance
    return round(cv2.compareHist(hist1,hist2,3),3)

def getEntropy(img1,img2):
    im = cv2.absdiff(img1,img2)
    return sum(sum(sum(im)))

Extracting every features for each frame from each video

In [7]:
path=r'C:\Users\vikil\Desktop\upwork_images\original'
data = pd.DataFrame([])
counter=0
count=0
for files in os.listdir(path):
    video_path=path+'\\'+files
    cap = cv2.VideoCapture(video_path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print (counter,files," Number of frames: ", video_length)
    video=[]
    error=[]
    psnr_error=[]
    hist_error=[]
    ent=[]

    while True:
        ret, frame = cap.read()
        video.append(frame)
        count+=1
        if (count > (video_length-1)):
            cap.release()
            break
    for i in range(len(video)-1):
        mse=getMse(video[i],video[i+1])
        psnr=getPSNR(video[i],video[i+1])
        hist=getHistCompare(video[i],video[i+1])
        entropy=getEntropy(video[i],video[i+1])
        ent.append(entropy)
        error.append(mse)
        psnr_error.append(psnr)
        hist_error.append(hist)
        avg_error=round(sum(error)/len(error),3)
        avg_psnr_error=round(sum(psnr_error)/len(psnr_error),3)
        avg_hist_error=round(sum(hist_error)/len(hist_error),3)
        avg_entropy=round(sum(ent)/len(ent),3)
    data=data.append(pd.DataFrame({'Mean square error':avg_error,'PSNR':avg_psnr_error,'Histogram compare':avg_hist_error,'entropy':avg_entropy,'class':0},index=[0]),ignore_index=True)
    counter=counter+1


0 Bike No-Plate Move + Move.mp4  Number of frames:  259
1 Birds Move + Move.mp4  Number of frames:  209
2 Book Reading Move + Move.mp4  Number of frames:  193
3 Building Move + Still.mp4  Number of frames:  229
4 Calling Person(M) Still + Move.mp4  Number of frames:  197
5 Canal View Still + Move.mp4  Number of frames:  214
6 CCTV Bikes in Fog Still + Move.mp4  Number of frames:  149
7 CCTV Bolan in fog Still + Still.mp4  Number of frames:  116
8 CCTV Customer Footage.(E) (Still + Move) (3).mp4  Number of frames:  101
9 CCTV Customer Shoes(M) Still + Move.mp4  Number of frames:  108
10 CCTV Mobile Shop(E) Still + Still (1).mp4  Number of frames:  103
11 CCTV Mobile Shop(M) Still + Still.mp4  Number of frames:  115
12 CCTV Mobile Shop(N) Still + Still.mp4  Number of frames:  157
13 CCTV Moving Bike(E) Still + Move.mp4  Number of frames:  154
14 CCTV Moving Person(N) Still + Move.mp4  Number of frames:  121
15 CCTV Moving Vehicles(M) Still + Move.mp4  Number of frames:  165
16 CCTV Night

In [18]:
# data.to_csv(r'C:\Users\vikil\Desktop\upwork_images\csv files\original.csv',index=False)

# df=pd.read_csv(r'C:\Users\vikil\Desktop\upwork_images\csv files\original.csv')

In [19]:
# video_path=r'C:\Users\vikil\Desktop\upwork_images\original\Birds Move + Move.mp4'
# data=pd.DataFrame([])
# cap = cv2.VideoCapture(video_path)
# video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
# print (" Number of frames: ", video_length)
# video=[]
# error=[]
# psnr_error=[]
# hist_error=[]
# ent=[]

# while True:
#     ret, frame = cap.read()
#     video.append(frame)
#     count+=1
#     if (count > (video_length-1)):
#         cap.release()
#         break
# for i in range(len(video)-1):
#     mse=getMse(video[i],video[i+1])
#     psnr=getPSNR(video[i],video[i+1])
#     hist=getHistCompare(video[i],video[i+1])
#     entropy=getEntropy(video[i],video[i+1])
#     ent.append(entropy)
#     error.append(mse)
#     psnr_error.append(psnr)
#     hist_error.append(hist)
#     avg_error=round(sum(error)/len(error),3)
#     avg_psnr_error=round(sum(psnr_error)/len(psnr_error),3)
#     avg_hist_error=round(sum(hist_error)/len(hist_error),3)
#     avg_entropy=round(sum(ent)/len(ent),3)
# data=data.append(pd.DataFrame({'Mean square error':avg_error,'PSNR':avg_psnr_error,'Histogram compare':avg_hist_error,'entropy':avg_entropy,'class':0},index=[0]),ignore_index=True)
# counter=counter+1